In [2]:
import yaml
import numpy as np
import zstandard as zstd
import os
from TraceSimulator import TraceSimulator
from trace_IO import *

def read_yaml_to_dict(file_path):
    with open(file_path, 'r') as file:
        config_dict = yaml.safe_load(file)
    return config_dict

config = read_yaml_to_dict('../config.yaml')
ts = TraceSimulator(config)

In [3]:
energy = 0
n_sets = 1000
all_traces = []
for _ in range(n_sets):
    trace = ts.generate(E=energy, x=-40, y=80, z=-1800, no_noise=False, type_recoil='ER', quantize=True, phonon_only=False)
    
    all_traces.append(np.asarray(trace[0], dtype=np.float16))

save_traces_to_zstd(all_traces, f"/ceph/dwong/trigger_samplesMMC_noise_1000.zst")



In [4]:
loaded_traces = load_traces_from_zstd(f"/ceph/dwong/trigger_samples/NR/traces_energy_{energy}.zst", n_traces=n_sets)

In [ ]:
import numpy as np
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from tqdm import tqdm

# Placeholder for the trace generation and saving functions
# Assume ts.generate and save_traces_to_zstd are already defined

def generate_and_save_traces(energy):
    all_traces = []
    n_sets = 200
    for _ in range(n_sets):
        trace = ts.generate(E=energy, x=-40, y=80, z=-1800,
                            no_noise=False, type_recoil='ER',
                            quantize=True, phonon_only=False)
        all_traces.append(np.asarray(trace[0], dtype=np.float16))

    output_path = Path(f"/ceph/dwong/trigger_samples/test0/traces_energy_{energy}.zst")
    save_traces_to_zstd(all_traces, output_path)


def main():
    energy_values = list(range(5, 151, 5))
    max_threads = 10

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(generate_and_save_traces, energy): energy for energy in energy_values}
        for _ in tqdm(as_completed(futures), total=len(futures), desc="Generating Traces"):
            pass


if __name__ == "__main__":
    main()


Generating Traces:  33%|███▎      | 10/30 [07:10<03:03,  9.18s/it]  

In [ ]:
plot_trace_set(trace_set)